https://www.youtube.com/watch?v=PuZY9q-aKLw


In [1]:
# Main imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [2]:
# load data
company = 'FB'

start = dt.datetime(2012, 1, 1)
end = dt.datetime(2020, 1, 1)

data = web.DataReader(company, 'yahoo', start, end)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data =  scaler.fit_transform(data['Close'].values.reshape(-1,1))

prediction_days = 60

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [ ]:
# Build model

model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1)) #predicts the next closing 

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

Epoch 1/25
59/59 [==============================] - 4s 66ms/step - loss: 0.0196:
Epoch 2/25
59/59 [==============================] - 4s 71ms/step - loss: 0.0036
Epoch 3/25
59/59 [==============================] - 4s 73ms/step - loss: 0.0038: 0s - los
Epoch 4/25
59/59 [==============================] - 4s 72ms/step - loss: 0.0033
Epoch 5/25
59/59 [==============================] - 4s 68ms/step - loss: 0.0031
Epoch 6/25
59/59 [==============================] - 4s 74ms/step - loss: 0.0034
Epoch 7/25
59/59 [==============================] - 4s 72ms/step - loss: 0.0038
Epoch 8/25
59/59 [==============================] - 4s 69ms/step - loss: 0.0029
Epoch 9/25
59/59 [==============================] - 4s 69ms/step - loss: 0.0027
Epoch 10/25
59/59 [==============================] - 4s 71ms/step - loss: 0.0031
Epoch 11/25
59/59 [==============================] - 4s 72ms/step - loss: 0.0028
Epoch 12/25
59/59 [==============================] - ETA: 0s - loss: 0.002 - 4s 70ms/step - loss: 0.0027
Ep

In [ ]:
# Load test data
test_start = dt.datetime(2020, 1, 1)
test_end = dt.datetime.now()

In [ ]:
test_data = web.DataReader(company, 'yahoo', test_start, test_end)
actual_prices = test_data['Close'].values

In [ ]:
total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)

In [ ]:
model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.transform(model_inputs)

In [ ]:
# Make predictions on test data

x_test = []

for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x, 0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

prediction_prices = model.predict(x_test)
prediction_prices = scaler.inverse_transform(prediction_prices)

In [ ]:
# Plot the test predictions

plt.plot(actual_prices, color='black', label=f'Actual {company} Price')
plt.plot(prediction_prices, color='green',label=f'Predicted {company} Price')
plt.title(f'{company} Share Price')
plt.xlabel('Time')
plt.ylabel(f'{company} Share Price')
plt.legend
plt.show()

In [ ]:
real_data = [model_inputs[len(model_inputs)+ 1 - prediction_days:len(model_inputs+1), 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1],1))

In [ ]:
prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f'Prediction: {prediction}')